In [1]:
from llmcoder import LLMCoder

In [2]:
llmcoder = LLMCoder(
    # analyzers=["mypy_analyzer_v1", "signature_analyzer_v1"],
    analyzers=["signature_analyzer_v1"],
    max_iter=5,
    feedback_variant="coworker",
    n_procs=3,
    min_score=5,
    backtracking=False, 
    combining=True,
    verbose=True
    )

In [3]:
initial_code = '''# Import random forest regressor
'''

In [4]:
old_code = '''    def _get_completions_for(
            self,
            conversation: Conversation,
            model: str = 'gpt-3.5-turbo',
            temperature: float = 0.7,
            n: int = 1,
            max_retries: int = 5,
            delta_temperature: float = 0.2,
            max_temperature: float = 2,
            factor_n: int = 2,
            max_n: int = 32) -> None:
        """
        Use OpenAI's API to get completion(s) for the user's code for a given conversation

        Parameters
        ----------
        conversation: Conversation
            Tuple in the priority queue. Contains the completion/code over which the model will complete.
        model : str, optional
            The model to use for the completion, by default 'gpt-3.5-turbo'
        temperature : float, optional
            The temperature to use for the completion, by default 0.7
        n : int, optional
            The number of choices to generate, by default 1
        max_retries : int, optional
            The maximum number of retries to get a valid completion, by default 5
        delta_temperature : float, optional
            The amount to increase the temperature in case of repeated mistakes, by default 0.2
        max_temperature : float, optional
            The maximum temperature to use, by default 2
        factor_n : int, optional
            The factor to increase the number of choices in case of repeated mistakes, by default 2
        max_n : int, optional
            The maximum number of choices to use, by default 32
        """
        total_generate_candidates = 0

        # Get the completions from OpenAI's API
        candidates = self.client.chat.completions.create(
            messages=conversation.messages,
            model=model,
            temperature=temperature,
            n=n)  # type: ignore

        total_generate_candidates += len(candidates.choices)

        # Count the number of tokens generated
        self.n_tokens_generated += sum([len(self.encoder.encode(message.message.content)) for message in candidates.choices])

        # Filter out completions that are repetitions of previous mistakes
        valid_choices = [completion for completion in candidates.choices if not self._is_bad_completion(completion.message.content)]
        print("[ME] The number of valid choices is: ", len(valid_choices))
        # Remove duplicates
        valid_unique_contents = list(set([choice.message.content for choice in valid_choices]))
        print("[ME] The number of valid unique contents is: ", len(valid_unique_contents))

        # If all completions are repetitions of previous mistakes, increase the temperature and the number of choices until we get a valid completion
        increased_temperature = temperature
        increased_n = n
        repetition = 0

        while len(valid_unique_contents) < n and repetition < max_retries:
            if self.verbose:
                print(f"[LLMcoder] Found {total_generate_candidates - len(valid_choices)} repeated mistakes, {len(valid_choices) - len(valid_unique_contents)} duplicates. Increasing temperature to {increased_temperature:.1f} and number of choices to {increased_n}... [repetition {repetition + 1}/{max_retries}]")

            # Sample new candidates
            candidates = self.client.chat.completions.create(
                messages=conversation.messages,
                model=model,
                temperature=increased_temperature,
                n=increased_n)  # type: ignore

            total_generate_candidates += len(candidates.choices)

            # Add the new completions to the list of valid completions
            # Filter out completions that are repetitions of previous mistakes
            valid_choices += [completion for completion in candidates.choices if not self._is_bad_completion(completion.message.content)]

            # Remove duplicates
            valid_unique_contents = list(set([choice.message.content for choice in valid_choices]))

            increased_temperature = min(max_temperature, increased_temperature + delta_temperature)
            increased_n = min(max_n, increased_n * factor_n)
            repetition += 1

        # If we still do not have valid choices, abort
        if len(valid_unique_contents) == 0 and repetition >= max_retries:
            if self.verbose:
                print("[LLMcoder] Could not generate valid completions. Aborting...")
            return None

        # Write on file the valid choices
        if self.conversation_file is not None:
            with open(self.conversation_file, "a") as file:
                file.write(f'[LLMcoder] The {len(valid_unique_contents)} valid unique contents are: \n')
                '''

In [5]:
llmcoder.conversations[0].messages[0]["content"]

"You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.\nYou are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.\nPrecisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.\nYour code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.\nYour unique capability is to provide completions without altering, repeating, or commenting on the original code.\nYou offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.\nThis approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven compl

In [6]:
llmcoder._step(initial_code, 0.7, 0, n=3)

[LLMcoder] Choosing conversation R with score 0
[LLMCoder] The number of valid choices is:  3
[LLMCoder] The number of valid unique contents is:  1
[LLMcoder] Found 0 repeated mistakes, 2 duplicates. Increasing temperature to 0.7 and number of choices to 3... [repetition 1/5]
[LLMcoder] Found 0 repeated mistakes, 5 duplicates. Increasing temperature to 0.9 and number of choices to 6... [repetition 2/5]
[LLMcoder] Found 0 repeated mistakes, 11 duplicates. Increasing temperature to 1.1 and number of choices to 12... [repetition 3/5]
[LLMcoder] Found 0 repeated mistakes, 23 duplicates. Increasing temperature to 1.3 and number of choices to 24... [repetition 4/5]
[LLMcoder] Found 0 repeated mistakes, 47 duplicates. Increasing temperature to 1.5 and number of choices to 32... [repetition 5/5]
[LLMcoder] Analyzing completion...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[LLMc

In [7]:
print(f'Number of conversations: {len(llmcoder.conversations)}')

Number of conversations: 1


In [8]:
# Print paths of all conversations in the Prioirtiy Queue
for i, conversation in enumerate(llmcoder.conversations):
    print(f'Conversation {i} path: {conversation.path}')

Conversation 0 path: ['R', 0]


In [9]:
most_promising_conversation = llmcoder.conversations.pop(temperature=0)
    
if llmcoder.verbose:
    print(f'[LLMcoder] Choosing conversation {"-".join(str(node) for node in most_promising_conversation.path)} with score {round(most_promising_conversation.score, 2)}')

# If there is feedback available from previous analyses, add it to the prompt
if len(most_promising_conversation.analyses) > 0:
    feedback_prompt = llmcoder._feedback_prompt_template(
        [str(result['message']) for result in most_promising_conversation.analyses[-1].values()
            if (not result['pass'] or result['type'] == "info")])
    print(f'Feedback prompt: {feedback_prompt}')
    
# If there is not feedback available, the prompt will just be the user's code
else:
    feedback_prompt = ""

# Add the prompt to the messages
most_promising_conversation.add_message({'role': 'user', 'content': feedback_prompt + initial_code})

# Get new completions and add them to the priority queue
llmcoder._get_completions_for(most_promising_conversation, llmcoder.model_feedback, 0.7, n=3)

# Choose the second highest-scored conversation from the priority queue
llmcoder._get_completions_for(llmcoder.conversations.get_second_best_conversation(), llmcoder.model_feedback, 0.7, n=3)
# Choose the best children of the second highest-scored conversation from the priority queue and combine it with the first
second_most_promising_conversation = llmcoder.conversations.get_second_best_conversation()
# Combine and create an optimal completion
llmcoder._combine(most_promising_conversation, second_most_promising_conversation, n=3)

[LLMcoder] Choosing conversation R-0 with score 0
Feedback prompt: [INST]


Fix, improve and rewrite your completion for the following code:
[/INST]

[LLMCoder] The number of valid choices is:  3
[LLMCoder] The number of valid unique contents is:  1
[LLMcoder] Found 0 repeated mistakes, 2 duplicates. Increasing temperature to 0.7 and number of choices to 3... [repetition 1/5]
[LLMcoder] Found 0 repeated mistakes, 5 duplicates. Increasing temperature to 0.9 and number of choices to 6... [repetition 2/5]
[LLMcoder] Found 0 repeated mistakes, 11 duplicates. Increasing temperature to 1.1 and number of choices to 12... [repetition 3/5]
[LLMcoder] Found 0 repeated mistakes, 23 duplicates. Increasing temperature to 1.3 and number of choices to 24... [repetition 4/5]
[LLMcoder] Found 0 repeated mistakes, 47 duplicates. Increasing temperature to 1.5 and number of choices to 32... [repetition 5/5]
[LLMcoder] Analyzing completion...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running

In [10]:
combining_prompt = llmcoder._combining_prompt_template(most_promising_conversation, llmcoder.conversations.get_second_best_conversation(),
                                                               # If there is feedback available from previous analyses, add it to the prompt
                                                               [str(result['message']) for result in most_promising_conversation.analyses[-1].values()
                                                                if (not result['pass'] or result['type'] == "info")],
                                                               [str(result['message']) for result in llmcoder.conversations.get_second_best_conversation().analyses[-1].values()
                                                                if (not result['pass'] or result['type'] == "info")])
most_promising_conversation.add_message({'role': 'user', 'content': combining_prompt})
print(f'[LLMcoder] {most_promising_conversation.messages}')

[LLMcoder] [{'role': 'system', 'content': "You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.\nYou are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.\nPrecisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.\nYour code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.\nYour unique capability is to provide completions without altering, repeating, or commenting on the original code.\nYou offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.\nThis approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing th

In [ ]:
llmcoder._get_completions_for(most_promising_conversation, llmcoder.model_feedback, temperature=0.7, n=n)
llmcoder._get_best_completion(llmcoder.conversations)

In [10]:
new_prompt = "this is a new prompt"
new_conversation = llmcoder.conversations.get_second_best_conversation()
new_conversation.add_message({'role': 'user', 'content': new_prompt})
new_conversation.get_last_message()

'this is a new prompt'

In [14]:
print(len(llmcoder.conversations))

9


In [13]:
with open(llmcoder.conversation_file, "a") as file:
    for c in llmcoder.conversations.queue:
        print(f'[LLMcoder] {c.messages}')

[LLMcoder] [{'role': 'system', 'content': "You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.\nYou are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.\nPrecisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.\nYour code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.\nYour unique capability is to provide completions without altering, repeating, or commenting on the original code.\nYou offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.\nThis approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing th

In [9]:
completion = llmcoder.complete(initial_code, n=3)

[LLMcoder] Creating first completions...
[LLMcoder] Choosing conversation R with score 0
[LLMCoder] The number of valid choices is:  3
[LLMCoder] The number of valid unique contents is:  3
[LLMcoder] Analyzing 3 completions...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[Signatures] No problematic functions or classes found in the context.
[LLMcoder] Have 3 conversations:
[LLMcoder] Passing   Score     Prob      Path
[LLMcoder] True      0         0.3333    ['R', 0]
[LLMcoder] True      0         0.3333    ['R', 1]
[LLMcoder] True      0         0.3333    ['R', 2]
[LLMcoder] First completion is correct. Stopping early...


In [8]:
from llmcoder.conversation import Conversation, PriorityQueue

In [9]:
from llmcoder.utils import get_conversations_dir, get_openai_key, get_system_prompt, get_system_prompt_dir, get_combining_prompt, get_combining_prompt_dir

In [11]:
system_prompt = get_system_prompt()
system_prompt

"You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.\nYou are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.\nPrecisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.\nYour code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.\nYour unique capability is to provide completions without altering, repeating, or commenting on the original code.\nYou offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.\nThis approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven compl

In [12]:
combining_prompt = get_combining_prompt()
combining_prompt

'Create an optimal completion by combining the previous two code snippets and fixing their errors. Leverage algorithmic improvements, reduce redundancy, and extract the best ideas to fix the code according to the message errors. Try to boost efficiency in a concise and maintainable manner. The code snippets are followed by a list of errors. Your optimal solution should, after combining / generating an alternative completion, not cause the errors indicated after "Errors first completion" or "Errors second completion" (if any).'

In [ ]:
conversations = PriorityQueue(
            Conversation(
                score=0,  # Score does not matter here because we pop the conversation with the highest score anyway
                messages=[{
                    "role": "system",
                    "content": system_prompt}]),
            backtracking= True)


In [11]:
from llmcoder.conversation import Conversation, PriorityQueue
def _combine(self, conversation1: Conversation, conversation2: Conversation, n: int = 1) -> str:
    """
    Combine two conversations and return the best completion from the Priority Queue

    Parameters
    ----------
    conversation1 : Conversation
        The first conversation
    conversation2 : Conversation
        The second conversation
    n : int, optional
        The number of choices to generate, by default 1

    Returns
    -------
    str
        The best completion
    """
    # Combine the two conversations
    combining_prompt = self._combining_prompt_template(conversation1, conversation2, 
                                                            # If there is feedback available from previous analyses, add it to the prompt
                                                            [str(result['message']) for result in conversation1.analyses[-1].values()
                                                            if (not result['pass'] or result['type'] == "info")],
                                                            [str(result['message']) for result in conversation2.analyses[-1].values()
                                                            if (not result['pass'] or result['type'] == "info")])
    conversation1.add_message({'role': 'user', 'content': combining_prompt})
    # Log the current combining prompt
    self._write_combining_prompt_file(combining_prompt)
    self._get_completions_for(conversation1, self.model_feedback, temperature=0.7, n=2)        
    final_completion = self._get_best_completion(self.conversations)
    final_completion

In [12]:
llmcoder.conversations[0].messages[1]["content"]

'    def _get_completions_for(\n            self,\n            conversation: Conversation,\n            model: str = \'gpt-3.5-turbo\',\n            temperature: float = 0.7,\n            n: int = 1,\n            max_retries: int = 5,\n            delta_temperature: float = 0.2,\n            max_temperature: float = 2,\n            factor_n: int = 2,\n            max_n: int = 32) -> None:\n        """\n        Use OpenAI\'s API to get completion(s) for the user\'s code for a given conversation\n\n        Parameters\n        ----------\n        conversation: Conversation\n            Tuple in the priority queue. Contains the completion/code over which the model will complete.\n        model : str, optional\n            The model to use for the completion, by default \'gpt-3.5-turbo\'\n        temperature : float, optional\n            The temperature to use for the completion, by default 0.7\n        n : int, optional\n            The number of choices to generate, by default 1\n       

In [13]:
llmcoder._combine(llmcoder.conversations[0], llmcoder.conversations[1], n=3)

[LLMCoder] The number of valid choices is:  3
[LLMCoder] The number of valid unique contents is:  2
[LLMcoder] Found 0 repeated mistakes, 1 duplicates. Increasing temperature to 0.7 and number of choices to 3... [repetition 1/5]
[LLMcoder] Found 3 repeated mistakes, 1 duplicates. Increasing temperature to 0.9 and number of choices to 6... [repetition 2/5]
[LLMcoder] Found 8 repeated mistakes, 2 duplicates. Increasing temperature to 1.1 and number of choices to 12... [repetition 3/5]
[LLMcoder] Analyzing 3 completions...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[Signatures] No problematic functions or classes found in the 

In [14]:
llmcoder.conversations[0].messages[2]["content"]

'from sklearn.ensemble import RandomForestRegressor'

In [12]:
from llmcoder.conversation import Conversation, PriorityQueue
conversation1 = Conversation(-2, [
                {'role': 'pytest', 'content': 'Test system message'},
                {'role': 'pytest', 'content': 'Test user message'}
            ], [
                {
                    "analyzer1": {"type": "critical", "pass": False},
                    "analyzer2": {"type": "critical", "pass": False},
                }
            ])
conversation2 = Conversation(-1, [
        {'role': 'pytest', 'content': 'Test system message'},
        {'role': 'pytest', 'content': 'Better Test message'}
    ], [
        {
            "analyzer1": {"type": "critical", "pass": False},
            "analyzer2": {"type": "critical", "pass": False},
        }
    ])

In [13]:
conversation1.analyses

[{},
 {'analyzer1': {'type': 'critical', 'pass': False},
  'analyzer2': {'type': 'critical', 'pass': False}}]